In [1]:
from SimEx import *

/usr/lib64/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)


initializing ocelot...



In [2]:
from SimExSLURM import SLURMSubmitter

In [3]:
!ls *.ipynb
!ls SimEx*.py

Demo.ipynb  SimEx_Test.ipynb
SimExSLURM.py


In [4]:
from SimEx import *
import SimEx
from IPython.display import display
from ipywidgets import widgets

In [5]:
beam = PhotonBeamParameters(
    photon_energy = 4972.0*electronvolt,  # photon energy in eV
    beam_diameter_fwhm=1.3e-7*meter,         # focus diameter in m
    pulse_energy=0.45e-3*joule,                   # pulse energy in J
    photon_energy_relative_bandwidth=0.003, # relative bandwidth dE/E
    divergence=0.0*radian,                         # Beam divergence in rad
    photon_energy_spectrum_type='tophat',   # Spectrum type. Acceptable values are "tophat", "SASE", and "twocolor")
    )

In [6]:
parameters = CrystFELPhotonDiffractorParameters(
    sample='5udc.pdb',                    # Looks up pdb file in cwd, if not found, queries from RCSB pdb mirror.
    uniform_rotation=True,                # Apply random rotation
    number_of_diffraction_patterns=1,     #
    powder=False,                         # Set to True to create a virtual powder diffraction pattern (unested) 
    intensities_file=None,                # File that contains reflection intensities. If set to none, use uniform intensity distribution
    crystal_size_range=[1e-7,1e-7],       # Range ([min,max]) in units of metres of crystal size.
    poissonize=False,                     # Set to True to add Poisson noise.
    number_of_background_photons=0,       # Change number to add uniformly distributed background photons.
    suppress_fringes=False,               # Set to True to suppress side maxima between reflection peaks.
    beam_parameters=beam,                 # Beam parameters object from above
    detector_geometry='simple.geom',      # External file that contains the detector geometry in CrystFEL notation.
    )


Checking presence of 5udc.pdb. Will query from PDB if not found in $PWD.
Sample path is set to 5udc.pdb.


In [7]:
diffractor = CrystFELPhotonDiffractor(
    parameters=parameters,      
    output_path="xstal_diffr")  # Output will go to a new dir "xstal_diffr/" in the current directory.'

In [8]:
diffractor.parameters.nodes_per_task = 1
diffractor.parameters.gpus_per_task = 0
diffractor.parameters.cpus_per_task = 1

In [9]:
display(SLURMSubmitter(diffractor))

SLURMSubmitter(children=(HBox(children=(Label(value='Partition'), Dropdown(options=('all', 'maxwell', 'cfel', …

Submitted batch job 978442
Submitted batch job 978443
Submitted batch job 978444
Submitted batch job 978445
Submitted batch job 978446


In [10]:
def checkjob(m):
    stat = !squeue -u reppinjo
    html1.value = '{} {} {} {} {} {} {} {}'.format(*(stat[0].lower().split()))
    html2.value = '<br/>'.join(stat[1:])
    

In [11]:
w = widgets.VBox()
button = widgets.Button(description='Check SLURM jobs')
html1 =  widgets.HTML('HEADER')
html2 =  widgets.HTML('job status')
w.children = [button, html1, html2]
w.children[0].on_click(checkjob)
display(w)